Examine sensitivity of current best model (Gear-Specific, Multi-Window 
Logistic Model with is-daylight) with respect to degredation of the AIS
data. 

We train as normal, then progessively randomly dropout more and more of
the training data and see how that effects the resulting reported
accuracies.

In [1]:
%matplotlib inline
from vessel_scoring import data, utils
from vessel_scoring.models import train_model_on_data
from vessel_scoring.evaluate_model import evaluate_model, compare_models
from IPython.core.display import display, HTML, Markdown
import numpy as np
import sys
from sklearn import metrics

In [2]:
from vessel_scoring.logistic_model import LogisticModel

def make_model(seed=4321):
    return LogisticModel(colspec=dict(
        windows=[1800, 3600, 10800, 21600, 43200, 86400],
        measures=['measure_daylight', 'measure_speed']), order=6, random_state=seed)


In [3]:
def load_data(suffix='', seed=4321):
    # Data supplied by Kristina
    _, train_lline,  valid_lline, test_lline = data.load_dataset_by_vessel(
            '../datasets/kristina_longliner.measures{}.npz'.format(suffix), seed)
    _, train_trawl,  valid_trawl, test_trawl = data.load_dataset_by_vessel(
            '../datasets/kristina_trawl.measures{}.npz'.format(suffix), seed)
    _, train_pseine, valid_pseine, test_pseine = data.load_dataset_by_vessel(
            '../datasets/kristina_ps.measures{}.npz'.format(suffix), seed)

    # Slow transits (used to train models to avoid classifying slow transits as fishing)
    TRANSIT_WEIGHT = 10
    x_tran, xtrain_tran, xcross_tran, xtest_tran = data.load_dataset_by_vessel(
                        '../datasets/slow-transits.measures{}.npz'.format(suffix), even_split=False, seed=seed)
    xtrain_tran = utils.clone_subset(xtrain_tran, test_lline.dtype)
    xcross_tran = utils.clone_subset(xcross_tran, test_lline.dtype)
    xtest_tran = utils.clone_subset(xtest_tran, test_lline.dtype)
    train_tran = np.concatenate([xtrain_tran, xcross_tran] * TRANSIT_WEIGHT)

    train = {'longliner': np.concatenate([train_lline, valid_lline, train_tran]), 
                'trawler': np.concatenate([train_trawl, valid_trawl, train_tran]),
                'purse_seine': np.concatenate([train_pseine, valid_pseine, train_tran])}
    
    test = {'longliner': test_lline, 
            'trawler': test_trawl, 
            'purse_seine': test_pseine}
    
    return train, test

In [18]:
keep_probs = [1, 0.5, 0.2, 0.1, 0.05, 0.01, 0.02, 0.005]

train_data, test_data = load_data()

drop_test = {1: test_data}
for kp in keep_probs[1:]:
    suffix = '_{}'.format(str(kp).replace('.', ''))
    drop_test[kp] = load_data(suffix=suffix)[1]

for gear in ['purse_seine', 'trawler', 'longliner']:
    mdl = make_model()
    train_model_on_data(mdl, train_data[gear])
    
    display(HTML("<h2>{}</h2>".format(gear.replace('_', ' ').title())))
                
    predictions = []
    for kp in keep_probs:  

        td = drop_test[kp][gear]
            
        raw = mdl.predict_proba(td)[:,1]
        predictions.append((kp, raw, (raw > 0.5), td['classification'] > 0.5))

    lines = ["|Model|Recall|Precision|F1-Score|ROC-AUC|",
         "|-----|------|---------|--------|--------|"]
    for name, raw, pred, actual in predictions:
        fpr, tpr, thresholds = metrics.roc_curve(actual, raw)
        auc = metrics.auc(fpr, tpr)
        lines.append("|{}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|".format(name, 
                                            metrics.recall_score(actual, pred),
                                            metrics.precision_score(actual, pred), 
                                            metrics.f1_score(actual, pred),
                                            auc))

    display(Markdown('\n'.join(lines)))    
    display(HTML("<hr/>"))

Warning, insufficient items to sample, returning all
Warning, inufficient items to sample, returning 4430
Warning, inufficient items to sample, returning 2084
Warning, insufficient items to sample, returning all
Warning, inufficient items to sample, returning 2244
Warning, inufficient items to sample, returning 1084
Warning, insufficient items to sample, returning all
Warning, inufficient items to sample, returning 922
Warning, inufficient items to sample, returning 453
Warning, inufficient items to sample, returning fewer
Warning, insufficient items to sample, returning all
Warning, inufficient items to sample, returning 467
Warning, inufficient items to sample, returning 225
Warning, inufficient items to sample, returning fewer
Warning, insufficient items to sample, returning all
Warning, inufficient items to sample, returning 81
Warning, inufficient items to sample, returning fewer
Warning, insufficient items to sample, returning all
Warning, inufficient items to sample, returning f

|Model|Recall|Precision|F1-Score|ROC-AUC|
|-----|------|---------|--------|--------|
|1|0.72|0.21|0.33|0.93|
|0.5|0.62|0.22|0.32|0.90|
|0.2|0.55|0.17|0.26|0.89|
|0.1|0.48|0.21|0.29|0.85|
|0.05|0.44|0.17|0.25|0.88|
|0.01|0.17|0.14|0.15|0.83|
|0.02|0.15|0.10|0.12|0.82|
|0.005|0.21|0.30|0.25|0.87|

|Model|Recall|Precision|F1-Score|ROC-AUC|
|-----|------|---------|--------|--------|
|1|0.87|0.94|0.90|0.95|
|0.5|0.81|0.94|0.87|0.94|
|0.2|0.80|0.93|0.86|0.94|
|0.1|0.73|0.95|0.83|0.93|
|0.05|0.67|0.97|0.80|0.92|
|0.01|0.36|0.94|0.52|0.87|
|0.02|0.51|0.96|0.66|0.90|
|0.005|0.23|0.95|0.37|0.88|

|Model|Recall|Precision|F1-Score|ROC-AUC|
|-----|------|---------|--------|--------|
|1|0.90|0.94|0.92|0.95|
|0.5|0.90|0.96|0.93|0.95|
|0.2|0.87|0.96|0.92|0.94|
|0.1|0.89|0.99|0.94|0.98|
|0.05|0.79|0.97|0.87|0.94|
|0.01|0.47|0.98|0.64|0.88|
|0.02|0.64|0.97|0.77|0.91|
|0.005|0.41|0.97|0.58|0.87|